# This script generates a video of general hive visualisation

## Imports

In [1]:
import multiprocessing, cv2, os, sys
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
sys.path.append(os.path.abspath('ABCThermalPlots'))
sys.path.append(os.path.abspath('ABCImaging'))
from libvisu import *
from ABCImaging.libimage import fetchImagesPaths
from ABCImaging.VideoManagment.videolib import generateVideoFromList
from InfluxDBInterface.libdb import download_data_DB
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Set a wide width for display
pd.set_option('display.colheader_justify', 'center')  # Align column headers
from dask.distributed import Client
# Start a client using all CPUs of this machine
client = Client()
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
/opt/anaconda3/envs/imaging/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64000 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory '/var/folders/w2/m4mvm5pj37107n10n7fs6sd80000gr/T/dask-scratch-space/worker-10bj7rck', purging
INFO:distributed.diskutils:Found stale lock file and directory '/var/folders/w2/m4mvm5pj37107n10n7fs6sd80000gr/T/dask-scratch-space/worker-hyvnk4wh', purging
INFO:distributed.diskutils:Found stale lock file and directory '/var/folders/w2/m4mvm5pj37107n10n7fs6sd80000gr/T/dask-scratch-space/worker-nvf5jnql', purging
INFO:distributed.diskutils:Found stale lock file and directory '/var/folders/w2/m4mvm5pj37107n10n7fs6sd80

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:64000/status,
Dashboard: http://127.0.0.1:64000/status,Workers: 4
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64001,Workers: 0
Dashboard: http://127.0.0.1:64000/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:64012,Total threads: 3
Dashboard: http://127.0.0.1:64019/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:64004,


## Configuration

In [2]:
# ==== GENERAL SETTINGS ====
download_data = True    # Set to True to download data from InfluxDB, False to use data stored locally
hive_nb = 1             # Hive number to process (1 or 2 for OH, 3 for BH and 0 for debugging)
t_res = "5min"          # Image time resolution. Change only if images are not taken every minute. Use frame_drop to keep only some frames, not this.
rootpath_imgs = '/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/25.07-25.10_aSensing_OH/Images/'     # For images
first_dt = pd.to_datetime("250716-001000Z", format='%y%m%d-%H%M%S%z')
last_dt = pd.to_datetime("250728-090000Z", format='%y%m%d-%H%M%S%z')
frame_drop = 1          # We keep 1 frame every frame_drop frames. Put one to keep all frames.
verbose = True          # Set to True to print progress messages

# ==== LOCAL DATA CONFIGURATION ====
if not download_data:
    data_path = 'data/2025-10-06_13-18_influxdb_data.csv' # Path to the local .csv or .dat file containing the CO2, htr and tmp data

# ==== DATA DOWNLOAD CONFIGURATION ====
if download_data:
    bucket = 'a_sensing' # InfluxDB bucket to download data from

## Main code

### Image fetching

In [3]:
# Get the target dt (for which we need an image, data, etc.)
datetimes = pd.date_range(start=first_dt, end=last_dt, freq=t_res)
datetimes = datetimes[::frame_drop]

imgs_paths = fetchImagesPaths(rootpath_imgs, datetimes, hive_nb)
print(f"Found {len(imgs_paths)} images for hive {hive_nb} between {first_dt} and {last_dt}.")

Found 3563 images for hive 1 between 2025-07-16 00:10:00+00:00 and 2025-07-28 09:00:00+00:00.


### Htr, co2 and tmp data fetching

In [4]:
if download_data:
    data_res = int(t_res[0]) * 60
    filters = {'hive_num': str(hive_nb),
               'measurement': ['co2', 'htr', 'tmp']}
    hive_data = download_data_DB(bucket, first_dt, last_dt, data_res, filters, verbose=verbose) # Restricted to our time frame and hive
else:
    data_path = os.path.abspath(data_path)          # Absolute path to the data
    hive_data = extractData(data_path, hive_nb, datetimes, verbose=False) # Restricted to our time frame and hive
print(hive_data.head())

Executing query:

    from(bucket: "a_sensing")
      |> range(start: 2025-07-16T00:05:00Z, stop: 2025-07-28T09:00:00Z)
      |> filter(fn: (r) => r["_measurement"] == "co2" or r["_measurement"] == "htr" or r["_measurement"] == "tmp")
      |> filter(fn: (r) => r["hive_num"] == "1")
      |> aggregateWindow(every: 300s, fn: last, createEmpty: false)
      |> yield(name: "last")
    
                                    _start                    _stop              _value      _field    _measurement board_id geo_loc hive_num inhive_loc          mcu_uuid          phys_loc  rpi_num   serial_id    valid DeltaT actuator_instance content  filling_density
_time                                                                                                                                                                                                                                                                       
2025-07-16 00:10:00+00:00 2025-07-16 00:05:00+00:00 2025-07-28 09:00:00+00:0

### tmp fetching

In [5]:
upper, lower = generateThermalDF(hive_data)   # Only tmp data, in a format that can be used by ThermalFrame

# Seek the max and min values of the tmp data, in both upper and lower hives
# max_temp = max(upper.max().max(), lower.max().max())
# min_temp = min(upper.min().min(), lower.min().min())

max_temp = np.nanmax([upper.to_numpy(), lower.to_numpy()])
min_temp = np.nanmin([upper.to_numpy(), lower.to_numpy()])
print("Max temperature in the selected time range: ", max_temp)
print("Min temperature in the selected time range: ", min_temp)
print(lower)

Max temperature in the selected time range:  56.1484375
Min temperature in the selected time range:  22.109375
                              t00        t01        t02        t03        t04        t05        t06        t07        t08        t09        t10        t11        t12        t13        t14        t15        t16        t17        t18        t19        t20        t21        t22        t23        t24        t25        t26        t27        t28        t29        t30        t31        t32        t33        t34        t35        t36        t37        t38        t39        t40        t41        t42        t43        t44        t45        t46        t47        t48        t49        t50        t51        t52        t53        t54        t55        t56        t57        t58        t59        t60        t61        t62        t63   
_time                                                                                                                                                          

### Metabolic data fetching

In [6]:
# Now we fetch the metabolic data
co2_data =generateMetabolicDF(hive_data)
print(co2_data)

                              ul        ur         ll        lr   
_time                                                             
2025-07-16 00:10:00+00:00  6238.1110   9041.0   6830.2870   8037.0
2025-07-16 00:15:00+00:00  6929.9100   6506.0  20488.5700   3843.0
2025-07-16 00:20:00+00:00  6405.0330   7509.0  22285.7900   4101.0
2025-07-16 00:25:00+00:00  7737.3250   9772.0  16116.0700  10315.0
2025-07-16 00:30:00+00:00  8056.5940  11650.0   8251.3460  10773.0
...                              ...      ...         ...      ...
2025-07-28 08:40:00+00:00   659.0056    557.0    756.5037    619.0
2025-07-28 08:45:00+00:00   686.0430    603.0    679.8256    668.0
2025-07-28 08:50:00+00:00   684.1136    555.0    808.0074    613.0
2025-07-28 08:55:00+00:00   692.7924    597.0    768.2055    656.0
2025-07-28 09:00:00+00:00   710.4068    591.0    762.2069    662.0

[3563 rows x 4 columns]


### htr fetching

In [7]:
# Alternative used now:
upper_htr = hive_data[(hive_data["inhive_loc"] == "upper") & (hive_data["_measurement"] == "htr")]
lower_htr = hive_data[(hive_data["inhive_loc"] == "lower") & (hive_data["_measurement"] == "htr")]
# Drop the _measurement column
upper_htr = upper_htr.drop(columns=["_measurement"])
lower_htr = lower_htr.drop(columns=["_measurement"])
print(upper_htr) # TODO: it is missing columns. Check why, and how we can avoid it from breaking the automation part at the bottom of the nb !!
print(lower_htr)

                                    _start                    _stop             _value    _field  board_id geo_loc hive_num inhive_loc          mcu_uuid          phys_loc  rpi_num   serial_id    valid DeltaT actuator_instance content  filling_density
_time                                                                                                                                                                                                                                                     
2025-07-16 00:10:00+00:00 2025-07-16 00:05:00+00:00 2025-07-28 09:00:00+00:00   0.00000       obj   abc03    EPFL      1       upper    2191217016663093217343026   bass       1     /dev/ttyACM0  True    0.0          h03        Unknown      Unknown   
2025-07-16 00:10:00+00:00 2025-07-16 00:05:00+00:00 2025-07-28 09:00:00+00:00  28.63281  avg_temp   abc03    EPFL      1       upper    2191217016663093217343026   bass       1     /dev/ttyACM0  True    0.0          h05        Unknown      Unknown

### Plotting

In [ ]:
vmax = max_temp
vmin = min_temp
# For a random time frame, generate the image of the hive and the thermal plot side by side with matplotlib
# This is just to check that everything is working fine
plt.figure(figsize=(18, 12))
frame = 2068 # Frame idx within datetimes
dt = datetimes[frame]
_imgs_paths = imgs_paths.loc[dt].to_numpy()
imgs_names = [str(_imgs_paths[i]).split("/")[-1][:-4] for i in range(len(_imgs_paths))]
# Read the images using cv2
current_imgs = [cv2.imread(_imgs_paths[i], cv2.IMREAD_GRAYSCALE) if _imgs_paths[i] is not None else np.zeros((2592,4608), np.uint8) for i in range(len(_imgs_paths))]

# Thermal data
try:
    upper_tf = ThermalFrame(upper.loc[dt].to_numpy())
except (NoValidSensors, KeyError):
    upper_tf = None

try:
    lower_tf = ThermalFrame(lower.loc[dt].to_numpy())
except (NoValidSensors, KeyError):
    lower_tf = None

# Metabolic data
if dt in co2_data.index:
    metabolic_df = co2_data.loc[dt]
else:
    metabolic_df = None

# Lower heater
if dt in lower_htr.index:
    lower_htr_row = lower_htr.loc[dt]
else:
    lower_htr_row = None

# Upper heater
if dt in upper_htr.index:
    upper_htr_row = upper_htr.loc[dt]
else:
    upper_htr_row = None

hs_1 = Hive(dt, current_imgs, False, imgs_names, upper_tf, lower_tf, htr_upper=upper_htr_row, htr_lower=lower_htr_row, hive_nb=hive_nb)
contours = list(range(12, 37, 3))
snapshot = hs_1.snapshot(v_max=32, v_min=23, contours=contours, annotate_contours=True, annotate_names=False, thermal_transparency=0.25)
hs_1.ilastikSegmentHoney()
snapshot = hs_1.honeySnapshot(transparency=0.25,v_min=23,v_max=32, annotate_names=False)
plt.imshow(snapshot)
plt.axis('off') 
plt.show()
cv2.imwrite('Figure2.png', cv2.cvtColor(snapshot, cv2.COLOR_RGB2BGR))

AttributeError: 'Hive' object has no attribute 'honey_masks'

<Figure size 1800x1200 with 0 Axes>

### Automation

In [9]:
final_imgs = []
print("Generating frames...")
for dt in tqdm(datetimes):
    _imgs_paths = imgs_paths.loc[dt].to_numpy()
    imgs_names = [str(_imgs_paths[j]).split("/")[-1][:-4] for j in range(len(_imgs_paths))]
    # Read the images using cv2
    current_imgs = []
    for _img_path in _imgs_paths:
        if _img_path is not None:
            img = cv2.imread(_img_path, cv2.IMREAD_GRAYSCALE)
            current_imgs.append(img)
        else:
            current_imgs.append(np.zeros((2592,4608), np.uint8))
        
    # Thermal data
    try:
        upper_tf = ThermalFrame(upper.loc[dt].to_numpy())
    except (NoValidSensors, KeyError):
        upper_tf = None
    
    try:
        lower_tf = ThermalFrame(lower.loc[dt].to_numpy())
    except (NoValidSensors, KeyError):
        lower_tf = None

    # Metabolic data
    if dt in co2_data.index:
        metabolic_df = co2_data.loc[dt]
    else:
        metabolic_df = None

    # Lower heater
    if dt in lower_htr.index:
        lower_htr_row = lower_htr.loc[dt]
    else:
        lower_htr_row = None

    # Upper heater
    if dt in upper_htr.index:
        upper_htr_row = upper_htr.loc[dt]
    else:
        upper_htr_row = None

    hs = Hive(dt, current_imgs, False, imgs_names, upper_tf, lower_tf, metabolic_df, upper_htr_row, lower_htr_row, hive_nb=hive_nb)

    snapshot = hs.snapshot(v_max=max_temp, v_min=min_temp,contours=contours,annotate_contours=True, thermal_transparency=0.4)
    final_imgs.append(snapshot)

# Change frames from rgb to bgr
print("Convert frames to bgr...")
for i in tqdm(range(len(final_imgs))):
    final_imgs[i] = cv2.cvtColor(final_imgs[i], cv2.COLOR_RGB2BGR)

dest = "outputVideos/"
# Make this a global path
print("Saving video...")
generateVideoFromList(final_imgs, dest="outputVideos/", name=f"hive{hive_nb}_{first_dt.strftime('%y%m%d-%H%M%Z')}_{last_dt.strftime('%y%m%d-%H%M%Z')}", fps=10, grayscale=False)

# Cleanup step to release resources
multiprocessing.active_children()

Generating frames...


100%|██████████| 2264/2264 [30:24<00:00,  1.24it/s]


Convert frames to bgr...


100%|██████████| 2264/2264 [01:42<00:00, 22.12it/s]


Saving video...


Writing video: 100%|██████████| 2264/2264 [06:34<00:00,  5.75frame/s]


[<SpawnProcess name='Dask Worker process (from Nanny)' pid=59617 parent=59577 started daemon>,
 <SpawnProcess name='Dask Worker process (from Nanny)' pid=59620 parent=59577 started daemon>,
 <SpawnProcess name='Dask Worker process (from Nanny)' pid=59619 parent=59577 started daemon>,
 <SpawnProcess name='Dask Worker process (from Nanny)' pid=59618 parent=59577 started daemon>]